<a href="https://colab.research.google.com/github/yezzzzin/LIEON_preprocessing/blob/main/vsa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opensmile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 996.2/996.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.7/141.7 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.4/138.4 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.4/167.4 kB 24.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for iso-639: filename=iso_639-0.4.5-py3-none-any.whl size=168840 sha256=cf5e00b1bc70c551ae41babcc3ecf5661cb76504fa145bde5b316fb846788666
  Stored in directory: /root/.cache/pip/wheels/d8/78/cc/5478ca3b1c3f602eae6f8cdbd78f909c0a0bfa0bbcb5c7771f
Successfully built iso-639


In [ ]:
# 필요한 라이브러리 및 모듈 import
import pandas as pd
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import opensmile

In [ ]:
# opensmile 라이브러리 초기화
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.GeMAPSv01b,
    feature_level=opensmile.FeatureLevel.Functionals
)

In [ ]:
# 라벨이 있는 CSV 파일 경로
labeled_csv_file = '/content/drive/MyDrive/labeled_data.csv'

In [ ]:
# 라벨이 있는 CSV 파일을 복제하여 라벨을 제거한 새로운 CSV 파일을 생성하는 함수
def remove_label_from_csv(input_csv_file, output_csv_file):
    # 입력 CSV 파일을 읽어옴
    labeled_data = pd.read_csv(input_csv_file)

    # 라벨 열 따로 저장
    labels = labeled_data['label']

    # 라벨 열을 제거하여 새로운 DataFrame 생성
    new_data = labeled_data.drop(columns=['label'])

    # 새로운 CSV 파일로 저장
    new_data.to_csv(output_csv_file, index=False)

    # 라벨을 따로 저장
    labels.to_csv('/content/drive/MyDrive/labels.csv', index=False)

In [ ]:
# 함수 호출하여 라벨이 없는 새로운 CSV 파일 생성
remove_label_from_csv(labeled_csv_file, '/content/drive/MyDrive/labeled_data_without_label.csv')

In [ ]:
# 라벨이 없는 CSV 파일 경로
csv_file_without_label = '/content/drive/MyDrive/labeled_data_without_label.csv'

In [ ]:
# CSV 파일로부터 WAV 파일로 변환하는 함수
def csv_to_wav(input_csv_file, output_wav_file):
    # CSV 파일 읽기
    df = pd.read_csv(input_csv_file)

    # DataFrame을 numpy 배열로 변환
    data = df.to_numpy()

    # WAV 파일로 변환
    librosa.output.write_wav(output_wav_file, data, sr=48000)

In [ ]:
# MFCC 추출 함수
def extract_mfcc(input_wav_file):
    # Load audio file
    y, sr = librosa.load(input_wav_file)

    # Extract MFCC features
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)

    return mfccs

# 시각화 함수
def visualize_mfcc(mfccs):
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(mfccs, x_axis='time')
    plt.colorbar()
    plt.title('MFCC')
    plt.tight_layout()
    plt.show()

In [ ]:
# CSV 파일로 변환하는 함수
def mfcc_to_csv(mfccs, output_csv_file):
    # DataFrame으로 변환
    df = pd.DataFrame(mfccs.T)

    # CSV 파일로 저장
    df.to_csv(output_csv_file, index=False)

In [ ]:
# 라벨을 다시 적용하는 함수
def reapply_labels(original_csv_file, labels_csv_file, output_csv_file):
    # 원본 CSV 파일과 라벨 파일 읽기
    original_df = pd.read_csv(original_csv_file)
    labels_df = pd.read_csv(labels_csv_file)

    # 라벨 열 추가
    original_df['label'] = labels_df['label']

    # 새로운 CSV 파일로 저장
    original_df.to_csv(output_csv_file, index=False)

In [ ]:
# 1. CSV 파일을 WAV 파일로 변환
output_wav_file = '/content/drive/MyDrive/labeled_data_without_label.wav'
csv_to_wav(csv_file_without_label, output_wav_file)

# 2. WAV 파일에서 MFCC 추출
mfccs = extract_mfcc(output_wav_file)

# 3. MFCC 시각화
visualize_mfcc(mfccs)

# 4. MFCC를 CSV 파일로 저장
output_mfcc_csv_file = '/content/drive/MyDrive/labeled_mfcc_data.csv'
mfcc_to_csv(mfccs, output_mfcc_csv_file)

# 5. 원래 라벨 파일 경로
labels_csv_file = '/content/drive/MyDrive/labels.csv'

# 라벨을 다시 적용한 CSV 파일 생성
reapplied_labels_csv_file = '/content/drive/MyDrive/labeled_mfcc_data_with_labels.csv'
reapply_labels(output_mfcc_csv_file, labels_csv_file, reapplied_labels_csv_file)

In [ ]:
# VSA 추출 함수
def extract_vsa(input_wav_file):
    # Load audio file
    y, sr = librosa.load(input_wav_file)

    # Process audio signal to extract VSA features
    vsa_features = smile.process_signal(y, sr)

    return vsa_features

In [ ]:
# VSA 추출
vsa_features = extract_vsa(output_wav_file)

In [ ]:
# VSA를 CSV 파일로 저장
output_vsa_csv_file = '/content/drive/MyDrive/labeled_vsa_data.csv'
mfcc_to_csv(vsa_features, output_vsa_csv_file)

In [ ]:
# VSA에 기존의 라벨을 다시 적용한 CSV 파일 생성
reapplied_labels_csv_file_vsa = '/content/drive/MyDrive/labeled_vsa_data_with_labels.csv'
reapply_labels(output_vsa_csv_file, labels_csv_file, reapplied_labels_csv_file_vsa)